In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from facenet_pytorch import InceptionResnetV1
from torchvision import models
from collections import Counter
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

In [2]:
# Define the transformation to normalize the data
# Define transformations for MNIST
transform = transforms.Compose([
     transforms.Resize((128,128)),  # Resize images to 32x32 to fit the CNN input size # Convert to grayscale
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Grayscale(1),
    transforms.Normalize((0.5), (0.5))  # Normalize
])

# Download and load the training and test sets
trainset = datasets.ImageFolder(
    root='../abc/train',
    transform = transform
)

testset = datasets.ImageFolder(
    root='../abc/test',
    transform = transform
)

In [3]:
# Accessing the mapping from folder names to class indices
class_names = testset.classes  # List of class names (folder names)
class_to_idx = testset.class_to_idx  # Dictionary mapping class name to index

In [4]:
class_to_idx

{'Abdullah Munir': 0,
 'Aizaz': 1,
 'Ali riaz': 2,
 'Anas': 3,
 'Anousha': 4,
 'Ayaz': 5,
 'Fahad': 6,
 'Faizan': 7,
 'Farhan': 8,
 'Hammad': 9,
 'Hannan': 10,
 'Haris': 11,
 'Hirar': 12,
 'Irfan': 13,
 'Kashmala': 14,
 'Khadija': 15,
 'Kumail': 16,
 'Maryam': 17,
 'Moiz': 18,
 'Moiz SaP': 19,
 'Moizullah': 20,
 'Mustafa': 21,
 'Noor': 22,
 'Obaid': 23,
 'Raheel': 24,
 'Saqib': 25,
 'Sufyan': 26,
 'Talal': 27,
 'Tayyaba': 28,
 'Usaid': 29,
 'Zeeshan': 30,
 'Zehra': 31,
 'basit': 32,
 'musaddiq': 33,
 'muzammil': 34,
 'osama': 35,
 'owais': 36,
 'rafay': 37,
 'shehroz': 38,
 'sohaib': 39,
 'vinay': 40,
 'zain': 41,
 'zainab': 42,
 'zamran': 43,
 'zayan': 44}

In [5]:
# Create DataLoaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=True)

In [5]:
#Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# Instantiate the model, define loss function and optimizer
model = InceptionResnetV1(classify=True, num_classes=46)
model.conv2d_1a = nn.Conv2d(
    in_channels=1,  # Change to 1 for grayscale images
    out_channels=32,  # Keep the same number of output channels
    kernel_size=3,
    stride=2,
    padding=0,
    bias=False
)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def save_checkpoint(model, optimizer, epoch, loss, file_path='checkpoint.pth'):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss
    }
    torch.save(checkpoint, file_path)
    print(f'Checkpoint saved at {file_path}')

In [8]:
def load_checkpoint(model, optimizer=None, file_path='checkpoint.pth'):
    checkpoint = torch.load(file_path)
    print("here")
    model.load_state_dict(checkpoint['model_state_dict'])
    
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    print(f'Checkpoint loaded from {file_path} (Epoch: {epoch}, Loss: {loss})')
    
    return model, optimizer, epoch, loss

Checkpoint loaded from checkpoint.pth (Epoch: 0, Loss: 0.8143842445935258)


In [21]:
# Load checkpoint if exists
start_epoch = 0
checkpoint_path = 'checkpoint.pth'
try:
    model, optimizer, start_epoch, _ = load_checkpoint(model, optimizer, checkpoint_path)
    start_epoch += 1  # Start training from the next epoch
except FileNotFoundError:
    print("No checkpoint found, starting training from scratch.")


num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    print("here")
    for images, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        
        images, labels = images.to(device), labels.to(device)
#         print(images.shape())
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize the model parameters
        
        running_loss += loss.item() * images.size(0)  # Update running loss
        
        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(trainset)
    epoch_train_accuracy = 100 * correct_train / total_train

    # Evaluate on test set
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    
    test_accuracy = 100 * correct_test / total_test

    # Print loss and accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_train_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%")
    
    # Save checkpoint
    save_checkpoint(model, optimizer, epoch, epoch_loss, file_path=checkpoint_path)

# Final model save after training completes
print("Final model saved as 'checkpoint.pth'")

Checkpoint loaded from checkpoint.pth (Epoch: 14, Loss: 0.004344968501995825)
here


Epoch 1/15: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:14<00:00,  3.64it/s]


Epoch [1/15]
Train Loss: 0.0043, Train Accuracy: 99.96%
Test Accuracy: 99.48%
Checkpoint saved at checkpoint.pth
here


Epoch 2/15: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:14<00:00,  3.66it/s]


Epoch [2/15]
Train Loss: 0.0019, Train Accuracy: 99.96%
Test Accuracy: 99.42%
Checkpoint saved at checkpoint.pth
here


...

Epoch [13/15]
Train Loss: 0.0001, Train Accuracy: 100.00%
Test Accuracy: 99.03%
Checkpoint saved at checkpoint.pth
here


Epoch 14/15: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:14<00:00,  3.67it/s]


Epoch [14/15]
Train Loss: 0.0002, Train Accuracy: 100.00%
Test Accuracy: 98.97%
Checkpoint saved at checkpoint.pth
here


Epoch 15/15: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:14<00:00,  3.66it/s]


Epoch [15/15]
Train Loss: 0.0002, Train Accuracy: 100.00%
Test Accuracy: 99.35%
Checkpoint saved at checkpoint.pth
Final model saved as 'checkpoint.pth'


In [9]:
# Instantiate the model
# model = MyModel()
model, optimizer, epoch, loss = load_checkpoint(model, optimizer=None, file_path='checkpoint.pth')
model.eval()


here
Checkpoint loaded from checkpoint.pth (Epoch: 14, Loss: 0.0001639607379082286)


InceptionResnetV1(
  (conv2d_1a): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1

In [11]:
class_names = trainset.classes

In [22]:
image = Image.open('img.png')
image = transform(image)
image = image.unsqueeze(0)
image = image.to(device)
with torch.no_grad():
        output = model(image)

_, predicted_class = torch.max(output, 1)
print(f'Predicted class: {predicted_class.item()}')

Predicted class: 1


In [10]:
import cv2
import torch
from torchvision import transforms
import numpy as np


# Capture video from webcam
cap = cv2.VideoCapture('vid3.mp4')  # Change to 'video.mp4' for video file
from facenet_pytorch import MTCNN

mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 300x300
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Grayscale(1),
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean and std for grayscale
])

while True:
    ret, frame = cap.read()
    print("here")
    if not ret:
        print("here")
        break
    # print("here")
    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces
    boxes, _ = mtcnn.detect(rgb_frame)
    print(boxes)
    # print("hello")
    
    if boxes is not None:
        for box in boxes:
#             print("hello")
            x1, y1, x2, y2 = map(int, box)
            
            # Extract face ROI
            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue
            
            try:
                # Apply transformations
                face_pil = Image.fromarray(face)  # Convert numpy array to PIL image
                face_transformed = transform(face_pil)  # Apply transformations
                face_transformed = face_transformed.unsqueeze(0)  # Add batch dimension
#                 print("hello")
            except Exception as e:
                print(f"Error during preprocessing: {e}")
#                 print("hello")
                continue
            
            # Move the tensor to the device (GPU/CPU)
            face_transformed = face_transformed.to(device)
            
            # Classify face
            with torch.no_grad():
                
                output = model(face_transformed)
                _, predicted = torch.max(output, 1)
                label = predicted.item()
            
            label_name = class_names[label]
            # Draw rectangle and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, str(label_name), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    # print("hello")
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release capture and close windows
cap.release()
cv2.destroyAllWindows()


here
[[371.6543273925781 132.37838745117188 465.11138916015625
  253.85226440429688]
 [22.69215202331543 69.1253433227539 104.49654388427734
  166.56210327148438]
 [375.83807373046875 13.260605812072754 404.9363098144531
  53.599029541015625]
 [557.7577514648438 -1.2948774099349976 585.9754638671875
  33.81245040893555]
 [254.09408569335938 6.322413444519043 277.1326904296875
  35.03812026977539]
 [154.71182250976562 17.03675651550293 175.8297576904297
  42.834835052490234]]
here
[[366.6255798339844 129.76683044433594 466.35333251953125
  255.55575561523438]
 [21.55864906311035 60.55766296386719 106.28187561035156
  168.85800170898438]
 [556.751220703125 -2.684908151626587 584.7658081054688 32.86768341064453]
 [377.3664245605469 15.912459373474121 402.8917236328125
  47.933834075927734]
 [252.65750122070312 6.234504699707031 276.0213317871094
  35.49906539916992]
 [153.7911376953125 16.040813446044922 173.62863159179688
  41.33671188354492]]
here
[[366.1958312988281 130.45761108398438 